# Opioid Addiction Project
## Notebook 06: Full Pipeline

This notebook is intended to simulate the full pipeline using .py modules, starting with submitting a user's input, preprocessing that input, then running it through prediction tasks, resulting in the final output of the user's prediction scores.

### W210, Capstone
Summer 2019

Team:  Cameron Kennedy, Aditi Khullar, Rachel Kramer, Sharad Varadarajan

# 0. Load Libraries and Set Global Variables
This analysis is performed in the cells below.

In [1]:
#Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.externals import joblib  #Used to save/load (pickle) models
from collections import defaultdict
import operator
from scipy import stats

#Custom data prep function used in both training and prediction 
import OpioidDataPrep as odp
import OpioidExecution as oe

#Set initial parameter(s)
pd.set_option('display.max_rows', 200)
pd.options.display.max_columns = 150
dataDir = './data/'

print('Pandas Version', pd.__version__)

Pandas Version 0.24.2


# 1. Simulate User Data

This section simulates the user entering various responses to questions. In the actual web tool, this input will come in the form of a dictionary, therefore this data should mimic that format.

In [6]:
#Simulate User Input

inputDict = dict()

#DEMOGRAPHICS
inputDict['NAME'] = 'Joe Capstone' #We will delete this column
inputDict['IRSEX'] = 1 #Gender: 'Male' or 'Female'
inputDict['IREDUHIGHST2'] = 10 #Education:
inputDict['AGE2'] = 16 #Age
    
#ALCOHOL
inputDict['IRALCRC'] = 2 #(Alcohol Recency)
inputDict['IRALCFY'] = 12 #(Alcohol Frequency Past Year)
inputDict['BNGDRKMON'] = 1 #(Binge drinking, past 30 days)
inputDict['HVYDRKMON'] = 1 #(Heavy drinking, past 30 days)
inputDict['IRALCAGE'] = 17 #(First time used alcohol)

#DRUGS + ALCOHOL
inputDict['TXYRRECVD2'] = 0 #(Ever alcohol/drug treatment, past yr)
inputDict['TXEVRRCVD2'] = 0 #(Ever alcohol/drug treatment, lifetime)


#TOBACCO
inputDict['IRCIGRC'] = 9 #(Tobacco Recency, incl. Never)
inputDict['CIGDLYMO'] = 91 #(Tobacco 30+ consecutive days)
inputDict['CIGAGE'] = 18 #(Tobacco Use Daily)
inputDict['PIPEVER'] = 2 #(Ever smoked a pipe)
inputDict['IRCGRRC'] = 4 #(Cigar recency)
inputDict['IRSMKLSSREC'] = 4 #(Smokeless tobacco recency)
    
#WEED
inputDict['IRMJRC'] = 1 #(Weed recency)
inputDict['MJYRTOT'] = 250 #(Weed days in past year)
inputDict['FUMJ18'] = 2 #(First used weed prior to age 18)
inputDict['FUMJ21'] = 2 #(First used weed prior to age 21)
    
#DEPRESSION
inputDict['ADDPREV'] = 2 #(Several days of depression)
inputDict['ADDSCEV'] = 2 #(Several days of discouraged about life)
    
##OTHER
inputDict['BOOKED'] = 1 #(Ever arrested & booked)

print(inputDict)

#Convert to dataframe

{'NAME': 'Joe Capstone', 'IRSEX': 1, 'IREDUHIGHST2': 10, 'AGE2': 16, 'IRALCRC': 2, 'IRALCFY': 12, 'BNGDRKMON': 1, 'HVYDRKMON': 1, 'IRALCAGE': 17, 'TXYRRECVD2': 0, 'TXEVRRCVD2': 0, 'IRCIGRC': 9, 'CIGDLYMO': 91, 'CIGAGE': 18, 'PIPEVER': 2, 'IRCGRRC': 4, 'IRSMKLSSREC': 4, 'IRMJRC': 1, 'MJYRTOT': 250, 'FUMJ18': 2, 'FUMJ21': 2, 'ADDPREV': 2, 'ADDSCEV': 2, 'BOOKED': 1}


# 1.b Web App Test

In [7]:
#Full Test
predProb, predPercentile, predFI = oe.generateReport(inputDict)
print('Predicted Probability: {:.0%}'.format(predProb))
print('Percentile of Predicted Probability: {:.0%}'.format(predPercentile))
print('Feature Importance (sorted low to high):')
predFI

C:\Users\camke\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Predicted Probability: 19%
Percentile of Predicted Probability: 54%
Feature Importance (sorted low to high):


{'AGE2': -0.07034271329944299,
 'FUMJ18': -0.05475428228451216,
 'IRALCAGE': -0.01582246322497748,
 'IREDUHIGHST2': -0.011477780569389818,
 'FUMJ21': -0.008926288815295105,
 'ADDPREV': -0.007881381349512806,
 'ADDSCEV': -0.005710015969690811,
 'IRCGRRC': -0.0048118618005509,
 'TXYRRECVD2': -0.002784593952038717,
 'TXEVRRCVD2': -0.0024843315189008465,
 'IRALCRC': -0.0019848097551581094,
 'CIGAGE': -0.0009058336820166155,
 'CIGDLYMO': -0.0008530496789806548,
 'IRSMKLSSREC': -0.0007598195700566725,
 'PIPEVER': -0.0005910768090568246,
 'IRCIGRC': 0.005773916708237226,
 'IRALCFY': 0.011330906871532287,
 'BOOKED': 0.0171481296606952,
 'BNGDRKMON': 0.017340029782525884,
 'HVYDRKMON': 0.020525789289245318,
 'IRSEX': 0.02216130375120372,
 'IRMJRC': 0.028511010662871428,
 'MJYRTOT': 0.03430067116484938}

# 2. Preprocess

In [4]:
#Convert inputs to list (pandas conversion to dataframe requires dict values to be lists)
for k in inputDict:
    inputDict[k] = [inputDict[k]]
print(inputDict)

#Convert dict to dataframe
df = pd.DataFrame.from_dict(inputDict)

#Run preprocessing on dataframe
df = odp.preprocess(df)
df

{'NAME': [['Joe Capstone']], 'IRSEX': [[1]], 'IREDUHIGHST2': [[10]], 'AGE2': [[16]], 'IRALCRC': [[2]], 'IRALCFY': [[12]], 'BNGDRKMON': [[1]], 'HVYDRKMON': [[1]], 'IRALCAGE': [[17]], 'TXYRRECVD2': [[0]], 'TXEVRRCVD2': [[0]], 'IRCIGRC': [[9]], 'CIGDLYMO': [[91]], 'CIGAGE': [[18]], 'PIPEVER': [[2]], 'IRCGRRC': [[4]], 'IRSMKLSSREC': [[4]], 'IRMJRC': [[1]], 'MJYRTOT': [[250]], 'FUMJ18': [[2]], 'FUMJ21': [[2]], 'ADDPREV': [[2]], 'ADDSCEV': [[2]], 'BOOKED': [[1]]}


TypeError: unsupported operand type(s) for -: 'int' and 'list'

In [ ]:
#Resort by column name (necessary to feed the model)
c = list(df.columns)
c.sort()
df = df[c]

# 3. Generate Predicitons

In [ ]:
#Load Models
model = joblib.load(dataDir+'modelXGB.model')
explainer = joblib.load(dataDir+'calibXGB.explainer')
probs = np.load(dataDir+'modelXGBPredProbs.npy')

In [ ]:
#Calculate Prediciton
predM = model.predict_proba(df)[0][1]
print('Predicted Probability: {:.0%}'.format(predM))

#Calculate Percentile
pct = stats.percentileofscore(probs, predM)/100
print('Percentile of Predicted Probability: {:.0%}'.format(pct))

#Generate shapley values from this row
shapVal = explainer.shap_values(df)

#Aggregate shapley values for one-hot vectors
shapDict = defaultdict(list) #Handy: creates blank list if key doesn't exist, or appends to it if it does.

#Get everything before the '_' character of each column name
#Then create the column index numbers for those keys 
#These numbers correspond to the locations in the shapley output array
for i, colName in enumerate(df.columns):
    shapDict[colName.split('_')[0]].append(i)
    
#Make a list of aggregated values shapley
for k in shapDict: #Loop through every key in the dict
    shapSum = 0.0 #Reset to 0
    for index in shapDict[k]: #Loop through every item in the key's value (a list of column indexes)
        shapSum += shapVal[0][index] #Add the value for each item
    shapDict[k] = shapSum #Replace the list with the aggregated shapley value (the sum of each individual value)

sortedShapDict = dict(sorted(shapDict.items(), key=operator.itemgetter(1)))
print('Feature Importance (sorted low to high):')
sortedShapDict